# Predicting Income From Census Information

In this lesson, we will build a machine learning model on some real data.

We will solve the following problem:

Based on a person's demographic information, can you predict whether they make more or less than 50K a year?

We will use this dataset: https://archive.ics.uci.edu/ml/datasets/Census+Income

It contains information like a person's age, country, education, and more.

# Download the Data

The website provides TWO sets of data. The first, called `adult.data` is what we will use as our training and validation sets.

The second, called `adult.test` is what we will use to evaluate the performance of our machine learning model.


**Exercise 1: Why can we NOT `adult.test` as training data in addition to evaluation data?**

In [ ]:
import requests

def fetch_data(url, filepath):
    r = requests.get(url)
    with open(filepath, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)

# Download adult.data and adult.test into into /tmp.
adult_data_path = '/tmp/adult.data'
fetch_data(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
    adult_data_path,
)

adult_test_path = '/tmp/adult.test'
fetch_data(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',
    adult_test_path,
)

# Read the Data

We will read the data using a Python library called Pandas, which can be used to manipulate CSV data
and also interoperates very nicely other Python libraries like numpy (numeric computation), scipy (scientific computation), scikit-learn (machine learning), matplotlib (plotting/visualization).

Basically, we read the CSV into a tabular data structure called a **DataFrame**.

In [ ]:
import pandas as pd
import numpy as np

"""
Taken from https://archive.ics.uci.edu/ml/datasets/Census+Income

age: continuous. 
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
fnlwgt: continuous. 
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
education-num: continuous. 
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
sex: Female, Male. 
capital-gain: continuous. 
capital-loss: continuous. 
hours-per-week: continuous. 
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

label_raw: >50K, <=50K. 
"""
col_names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education_num',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital_gain',
    'capital_loss',
    'hours_per_week',
    'native_country',
    'label',
]


adult_data_raw = pd.read_csv(adult_data_path, header=None, names=col_names)
adult_test_raw = pd.read_csv(adult_test_path, header=None, names=col_names)

adult_data_raw

# Preprocess the Data

Like all datasets, the census dataset needs some preprocessing to clean it.

In [ ]:
def preprocess(df_raw):
    # We re-write the label column so that it is 1 if the person makes > 50K and 0 if they
    # make <= 50K.
    LABEL_POSITIVE = '>50K'
    df = df_raw.copy()
    df.label = df.label.str.contains(LABEL_POSITIVE).astype(int)
    # Let's get rid of the fnlwgt column because it is meaningless.
    df = df.drop('fnlwgt', 1)
    return df

# The test data has junk on the first line.
adult_test_raw_drop = adult_test_raw.drop(0, axis=0)

adult_data = preprocess(adult_data_raw)
adult_test = preprocess(adult_test_raw_drop)

# Explore the Data

Before doing any machine learning, let's compute some basic statistics and visualizations from
the dataset.

After running the two cells below, here are some observations that we can make:

1. There are roughly 3x as many people who make <= 50K than > 50K
2. Almost all the people in the dataset are from the United States
3. People who work more hours per week tend to make more money
4. White people tend to make more money than black people


**
Exercise 2: Run the two cells below and make 4-5 more observations like the ones above.
**

In [ ]:
"""
Let's start by counting the number of examples (overall, positive, and negative)
and number of features.
"""

(num_examples, num_features) = adult_data.shape
(num_positive_examples, _) = (adult_data[adult_data.label == 1]).shape
(num_negative_examples, _) = (adult_data[adult_data.label == 0]).shape
print(
    """
    Number of Examples: {}
    Number of Features: {}
    Number of Positive Examples: {}
    Number of Negative Examples: {}
    """.format(
        num_examples,
        num_features - 1, # The label is not a feature so subtract if off.
        num_positive_examples,
        num_negative_examples,
    )
)

In [ ]:
"""
Let's make some bar charts and histograms to see how different features
are related to the label.
"""
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

def histograms(df, feature_name):
    colors = ['blue', 'green']
    labels = ['negative', 'positive']
    for i in (0, 1):
        values = df[df.label == i][feature_name]
        plt.hist(values, normed=1, facecolor=colors[i], alpha=0.3, label=labels[i])
    
    plt.legend(prop={'size': 10})

    plt.xlabel(feature_name)
    plt.ylabel('Frequency')
    plt.title('Histogram of feature = {}'.format(feature_name))
    plt.show()

def bar_charts(df, feature_name):
    colors = ['blue', 'green']
    labels = ['negative', 'positive']
    objects = sorted(df[feature_name].unique())
    for i in (0, 1):
        sub_df = df[df.label == i]
        N = sub_df.shape[0]
        values = sub_df.groupby([feature_name])[feature_name].count() / N
        values_as_dict = {k: v for (k, v) in values.items()}
        frequencies = [values_as_dict.get(k, 0) for k in objects]
        y_pos = np.arange(len(objects))
        plt.bar(y_pos, frequencies, facecolor=colors[i], align='center', alpha=0.3, label=labels[i])
        plt.xticks(y_pos, objects, rotation=90)
    plt.legend(prop={'size': 10})
    plt.ylabel('Frequency')
    plt.title('Bar chart of feature = {}'.format(feature_name))
    plt.show()
histograms(adult_data, 'age')
bar_charts(adult_data, 'education_num')
bar_charts(adult_data, 'marital_status')
bar_charts(adult_data, 'occupation')
bar_charts(adult_data, 'relationship')
bar_charts(adult_data, 'race')
bar_charts(adult_data, 'sex')
histograms(adult_data, 'capital_gain')
histograms(adult_data, 'capital_loss')
histograms(adult_data, 'hours_per_week')
bar_charts(adult_data, 'native_country')

# Prepare the Data for Machine Learning

Now that we've visualized the data, we want to feed it into a machine learning model.

However, recall that supervised machine learning models take the form ($\mathbf{x} \in \mathbb{R}^d$):

$$
f(\mathbf{x}) = y
$$

Where $y \in \{0, 1, ..., K\}$ for multi-classification ($K = 2$ is binary classification) and $y \in \mathbb{R}$ for regression.

**Exercise 3: What kind of problem are we solving here: Classification (in which case, what is $K$?) or regression?**

In order to do machine learning, we need a design matrix. That is, we need to turn our data into a matrix $X \in \mathbb{R}^{m \times d}$ where $m$ is the number of training examples and $d$ is the dimensionality of the feature vector.

The challenge, however, is that many of our feature values are not numbers. For example, one occupation in our dataset is called 'Tech-support', which is clearly not a number.

For continuous features, like  age and capital gain, this is trivial because they are already numbers.

For categorical features, like occupation and race, this is harder. Let's see how we can turn a categorical feature into a number.

## String Indexing and One-Hot Encoding
One approach you might take is **string indexing**. Basically, if there are $C$ categories, then we assign each category a numerical value between $1$ and $C$. For example, suppose there are three countries: "United States", "China", and "Russia". Then we can assign them numbers like "United States" = 1, "China" = 2, and "Russia" = 3.

This is still not right though, because it imposes an order on a categorical feature, which doesn't make sense. Consequently, we will use **one-hot encoding**. Basically, will create $C$ features: "is-united-states", "is-china", and "is-russia", each of which is binary.

With one-hot encoding, we can easily turn a categorical feature into a set of binary features and enable the machine learning model to learn them.

**
Exercise 3: What is the feature index for "education= Bachelors"? What is the feature value for index 75?
**

In [ ]:
# First, let's separate the label from the features.
def separate_label(df, label_col='label'):
    labels = df[label_col]
    df = df.drop(label_col, axis=1)
    return (df, labels)

adult_data_features, adult_data_labels = separate_label(adult_data)
adult_test_features, adult_test_labels = separate_label(adult_test)

# Now, let's one-hot encode categorical features
# DictVectorizer does one-hot encoding on all string columns, so we'll use that.
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer(sparse=False)
adult_data_matrix_unscaled = vectorizer.fit_transform(adult_data_features.to_dict('records'))
adult_test_matrix_unscaled = vectorizer.transform(adult_test_features.to_dict('records'))

# This maps feature name to feature index.
vectorizer.vocabulary_

## Scaling to Zero-Mean Unit-Variance

If you look at the matrix, you'll notice that there are some features with very large values. For example, the capital gains can be quite large because it is measured in dollars.

Such large-valued features can mess up the gradient descent algorithm because it will take large steps in that dimension, but small steps in other dimensions.

To mitigate this issue, we will normalize all the features so that they have zero mean and standard variance.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
adult_data_matrix = scaler.fit_transform(adult_data_matrix_unscaled)
adult_test_matrix = scaler.transform(adult_test_matrix_unscaled)

pd.DataFrame(adult_data_matrix)

# Machine Learning

Finally, our data is ready for machine learning.

Let's train a logistic regression model and evaluate its performance.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(adult_data_matrix, adult_data_labels)
adult_test_predictions = lr.predict(adult_test_matrix)

from sklearn.metrics import accuracy_score, precision_score, recall_score
def evaluate_predictions(y_true, y_pred):
    print('Accuracy = {}'.format(accuracy_score(y_true, y_pred)))
    precision, recall = precision_score(y_true, y_pred), recall_score(y_true, y_pred)
    print('Precision = {}'.format(precision))
    print('Recall = {}'.format(recall))
    print('F1 = {}'.format(2 * (precision * recall) / (precision + recall)))
    
evaluate_predictions(adult_test_labels, adult_test_predictions)

In [ ]:
# Let's see which features are the most important.
# Important features will have high magnitude weights.

feature_importance_pairs = []

feature_name_for_index = {v: k for (k, v) in vectorizer.vocabulary_.items()}
for feature_index in range(len(lr.coef_[0])):
    feature_name = feature_name_for_index[feature_index]
    coefficient = abs(lr.coef_[0][feature_index])
    importance = abs(coefficient)
    feature_importance_pairs.append((feature_name, importance))

# Features, sorted by decreasing importance.
sorted(feature_importance_pairs, key=lambda x: -x[1])

**Exercise 4: For four of the important features above, come up with an explanation for why they might be important. For example, if age should be an important feature because people gain more experience as they age and tend to make more money.**

**Exercise 5: The education = 'Assoc-acdm' feature seems like it should have high importance because it indicates soembody's education. However, its importance is very low. Why do you think this is?**

# Your Challenge

Now that we've trained a simple machine learning model and measured its performance, your goal is to train a better
machine learning model.

Here are some tricks for you to try:

1. Select hyperparameters for the Logistic Regression (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
2. Select those hyperparameters with search + cross validation (http://scikit-learn.org/stable/modules/grid_search.html)
3. Use a more powerful classifier like Support Vector Machines (http://scikit-learn.org/stable/modules/svm.html#classification), Decision Trees (http://scikit-learn.org/stable/modules/tree.html), and Gradient Boosted Decision Trees (http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html).
4. Drop features that you don't think will be useful


**Exercise 6: Beat the F1 score above**

In [ ]:
# YOUR CODE HERE